## 利用PyTorch从零实现Softmax回归

1. 导入数据

In [1]:
import torch
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [2]:
next(iter(train_iter))[0].shape # 生成一个批 256个样本的特征 [1]为label

torch.Size([256, 1, 28, 28])

2. 初始化参数w和b

In [3]:
# 对于softmax，需要将(1,28,28)的三维拉长为1*28*28的向量
num_inputs = 784
num_outputs = 10 # 数据共有十个类别

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True) # (784, 10)
b = torch.zeros(num_outputs, requires_grad=True) # 每个输出都要有个偏移 (10)


3. 定义softmax函数

In [4]:
def softmax(Z): # Z是个矩阵 (-1,10)
    Z_exp = torch.exp(Z)
    partition = Z_exp.sum(1, keepdim=True) # 因为共10列/类 故按行求和  并keepdim使能够下一行的广播
    return Z_exp / partition # 运用了广播机制，让每一行都除以分母 (batch_size, 784)与(batch_size,1)进行广播

In [5]:
# # 测试
# X = torch.normal(0,1,(2,5))
# x_prob = softmax(X)
# x_prob,x_prob.sum(1) # 根据输出可见，所有输出值都为正数，且行和均为1（即各个类别的概率合为1）

4. 定义softmax回归模型

In [6]:
def softmaxRegressionNet(X): # 为何不把W和b也设为参数呢？
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b) # X本来为(256,1,28,28) ?
    # reshape将每张图片的(1,28,28)展平为向量
    # X(256,784) * W(784,10) + b(10) -> (256,10)

In [7]:
# # 介绍一下花式索引
# a = torch.arange(12).reshape(3,4)
# a[[0,2], [2,3]] # 拿出第1行第3列 和 第3行第4类元素

y = torch.tensor([0,2])
y_hat = torch.tensor([[0.1,0.3,0.6], [0.3,0.2,0.5]]) # 输出的各样本各类别的概率
y_hat[[0,1], y] # 拿出第1行第1列 和 第2行第3类元素

tensor([0.1000, 0.5000])

5. 定义交叉熵损失函数 <br>
   交叉熵采用真实标签的预测概率的负对数似然

In [8]:
def cross_entropy(y_hat, y): # y_hat(256, 10), y(256, ) 对吗?
    return -torch.log(y_hat[range(len(y_hat)), y]) # 拿出对应真实标号的预测值

cross_entropy(y_hat, y) # y_hat(2,3) y(2,)  # 测试一下

tensor([2.3026, 0.6931])

6. 将预测类别与真实y进行比较

In [9]:
def accuracy(y_hat, y): # 计算预测正确的数量
    # 如果y_hat是矩阵，那么假定第二个维度存储每个类的预测分数
    # 使用argmax获得每行中最大元素的索引来获得预测类别
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1: # len(y_hat.shape)>1 说明是不止一维; and后面说明列数>1
        y_hat = y_hat.argmax(axis=1) 
    cmp = y_hat.type(y.dtype) == y # 现将y_hat的类型转为y的类型以防类型不一致，然后与y比较生成bool列表
    return float(cmp.type(y.dtype).sum())

accuracy(y_hat, y) / len(y) # 正确预测数量 / 总预测数量 = 正确概率

0.5

定义一个使用程序类Accumulator,用于对多个变量进行累加

In [10]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

评估数据迭代器data_iter在任意模型net的精度。

In [11]:
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集data_iter上模型net的精度"""
    if isinstance(net, torch.nn.Module): # 如果模型是torch nn实现的，转成评估模式(不计算梯度)
        net.eval()
    metric = Accumulator(2) # 累加器; 实例化
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel()) # 正确数量，总数量
    return metric[0] / metric[1] # 返回精度

evaluate_accuracy(softmaxRegressionNet, test_iter)
# 由于softmaxregressionNet模型使用随机的参数w和b初始化的，还未学习，接近随机猜测。那么10个类别随机猜测概率接近0.1

0.0745

7. Softmax回归的训练

In [ ]:
def train_epoch_ch3(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):  # 如果是nnModule(内置训练模型)
        net.train()
    metric = Accumulator(3)  # 长度为3的迭代器 累加所需信息
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)  # 交叉熵损失
        if isinstance(updater, torch.optim.Optimizer):  # 如果updater是内置的优化器
            updater.zero_grad()
            l.backward()  # 计算梯度
            updater.step()  # 更新参数
            metric.add(float(l)*len(y), accuracy(y_hat, y), y.size().numel())
        else: # 自己实现的
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())

    return metric[0] / metric[2], metric[1] / metric[2]
